In [142]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Unify") \
    .getOrCreate()

# I want to take all data (not per month)

- adding the month and year as columns
- convert trip distance to km instead of miles
- adding a field `amount-tip`
- unifying payment_type

## Test Data 01/2022

In [50]:
year = "2022"
month = "01"
df = spark.read.parquet(f"/taxi/by_month/{year}/{month}.parquet")

### adding month/year as columns

You can use `input_file_name` to get the filename of the dataframe. Here we have the month/year available

In [9]:
from pyspark.sql.functions import input_file_name

df.withColumn("filename", input_file_name()).show(2, False)

+-------------------+-------------------+---------------+-------------+------------+----------+------------+-------------------------------------------------------------------------------------------------------------------------------------+
|pickup_datetime    |dropoff_datetime   |passenger_count|trip_distance|payment_type|tip_amount|total_amount|filename                                                                                                                             |
+-------------------+-------------------+---------------+-------------+------------+----------+------------+-------------------------------------------------------------------------------------------------------------------------------------+
|2022-01-05 14:48:04|2022-01-05 14:55:39|1              |0.8          |1           |1.96      |11.76       |hdfs://bdlc-test.el.eee.intern:9000/taxi/by_month/2022/01.parquet/part-00031-153b5d72-b95a-46d1-b8b8-e7d88318a3c2-c000.snappy.parquet|
|2022-01-05 13:58:01|2022-01

In [10]:
from pyspark.sql.functions import input_file_name, split

df.withColumn("filename", split(input_file_name(), "/")).show(2, False)

+-------------------+-------------------+---------------+-------------+------------+----------+------------+----------------------------------------------------------------------------------------------------------------------------------------------+
|pickup_datetime    |dropoff_datetime   |passenger_count|trip_distance|payment_type|tip_amount|total_amount|filename                                                                                                                                      |
+-------------------+-------------------+---------------+-------------+------------+----------+------------+----------------------------------------------------------------------------------------------------------------------------------------------+
|2022-01-05 14:48:04|2022-01-05 14:55:39|1              |0.8          |1           |1.96      |11.76       |[hdfs:, , bdlc-test.el.eee.intern:9000, taxi, by_month, 2022, 01.parquet, part-00031-153b5d72-b95a-46d1-b8b8-e7d88318a3c2-c000.snappy.pa

In [17]:
from pyspark.sql.functions import input_file_name, split

df.withColumn("filename", split(input_file_name(), "/")).printSchema()

root
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- filename: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [33]:
df\
.withColumn("filename", split(input_file_name(), "/"))\
.selectExpr("*", "element_at(filename, -3) as year", "substring_index(element_at(filename, -2), '.' , 1) as month")\
.drop("filename")\
.show(2, False)

+-------------------+-------------------+---------------+-------------+------------+----------+------------+----+-----+
|pickup_datetime    |dropoff_datetime   |passenger_count|trip_distance|payment_type|tip_amount|total_amount|year|month|
+-------------------+-------------------+---------------+-------------+------------+----------+------------+----+-----+
|2022-01-05 14:48:04|2022-01-05 14:55:39|1              |0.8          |1           |1.96      |11.76       |2022|01   |
|2022-01-05 13:58:01|2022-01-05 14:05:21|1              |0.9          |1           |2.9       |12.7        |2022|01   |
+-------------------+-------------------+---------------+-------------+------------+----------+------------+----+-----+
only showing top 2 rows



### convert trip distance to km instead of miles

In [40]:
from pyspark.sql.functions import expr

df\
.withColumn("filename", split(input_file_name(), "/"))\
.selectExpr("*", "element_at(filename, -3) as year", "substring_index(element_at(filename, -2), '.' , 1) as month")\
.drop("filename")\
.withColumn("trip_distance", expr("trip_distance * 1.60934"))\
.show(2, False)

+-------------------+-------------------+---------------+------------------+------------+----------+------------+----+-----+
|pickup_datetime    |dropoff_datetime   |passenger_count|trip_distance     |payment_type|tip_amount|total_amount|year|month|
+-------------------+-------------------+---------------+------------------+------------+----------+------------+----+-----+
|2022-01-05 14:48:04|2022-01-05 14:55:39|1              |1.2874720000000002|1           |1.96      |11.76       |2022|01   |
|2022-01-05 13:58:01|2022-01-05 14:05:21|1              |1.448406          |1           |2.9       |12.7        |2022|01   |
+-------------------+-------------------+---------------+------------------+------------+----------+------------+----+-----+
only showing top 2 rows



### adding a field `amount-tip`

In [42]:
df\
.withColumn("filename", split(input_file_name(), "/"))\
.selectExpr("*", "element_at(filename, -3) as year", "substring_index(element_at(filename, -2), '.' , 1) as month")\
.drop("filename")\
.withColumn("trip_distance", expr("trip_distance * 1.60934"))\
.withColumn("trip_amount", expr("total_amount-tip_amount"))\
.show(2, False)

+-------------------+-------------------+---------------+------------------+------------+----------+------------+----+-----+-----------------+
|pickup_datetime    |dropoff_datetime   |passenger_count|trip_distance     |payment_type|tip_amount|total_amount|year|month|trip_amount      |
+-------------------+-------------------+---------------+------------------+------------+----------+------------+----+-----+-----------------+
|2022-01-05 14:48:04|2022-01-05 14:55:39|1              |1.2874720000000002|1           |1.96      |11.76       |2022|01   |9.8              |
|2022-01-05 13:58:01|2022-01-05 14:05:21|1              |1.448406          |1           |2.9       |12.7        |2022|01   |9.799999999999999|
+-------------------+-------------------+---------------+------------------+------------+----------+------------+----+-----+-----------------+
only showing top 2 rows



### unifying payment_type

In [47]:
df\
.withColumn("filename", split(input_file_name(), "/"))\
.selectExpr("*", "element_at(filename, -3) as year", "substring_index(element_at(filename, -2), '.' , 1) as month")\
.drop("filename")\
.withColumn("trip_distance", expr("trip_distance * 1.60934"))\
.withColumn("trip_amount", expr("total_amount-tip_amount"))\
.select("payment_type")\
.distinct()\
.show(20, False)

+------------+
|payment_type|
+------------+
|3           |
|1           |
|4           |
|2           |
|5           |
+------------+



In [54]:
# earlier data
old_df = spark.read.parquet(f"/taxi/by_month/2009/01.parquet")\
.withColumn("filename", split(input_file_name(), "/"))\
.selectExpr("*", "element_at(filename, -3) as year", "substring_index(element_at(filename, -2), '.' , 1) as month")\
.drop("filename")\
.withColumn("trip_distance", expr("trip_distance * 1.60934"))\
.withColumn("trip_amount", expr("total_amount-tip_amount"))


old_df.select("payment_type")\
.distinct()\
.show(20, False)

+------------+
|payment_type|
+------------+
|No Charge   |
|CASH        |
|Credit      |
|Cash        |
|Dispute     |
|CREDIT      |
+------------+



```
1 = Credit card 
2 = Cash
3 = No charge 
4 = Dispute
5 = Unknown 
6 = Voided trip
```

In [72]:
from pyspark.sql.functions import when, expr, col

old_df.withColumn('payment_type', \
                  when(col("payment_type") == "Credit", 1)\
                  .when(col("payment_type") == "CREDIT", 1)\
                  .when(col("payment_type") == "CASH", 2)\
                  .when(col("payment_type") == "Cash", 2)\
                  .when(col("payment_type") == "No Charge", 3)\
                  .when(col("payment_type") == "NO CHARGE", 3)\
                  .when(col("payment_type") == "Dispute", 4)\
                  .when(col("payment_type") == "DISPUTE", 4)\
                  .when(col("payment_type") == "Unknown", 5)\
                  .when(col("payment_type") == "UNKNOWN", 5)\
                  .when(col("payment_type") == "Voided Trip", 6)\
                  .when(col("payment_type") == "VOIDED TRIP", 6)\
                  .otherwise(expr("payment_type")))\
    .select("payment_type")\
    .distinct()\
    .show(20, False)


+------------+
|payment_type|
+------------+
|3           |
|1           |
|4           |
|2           |
+------------+



### All Together

In [78]:
df\
.withColumn("filename", split(input_file_name(), "/"))\
.selectExpr("*", "element_at(filename, -3) as year", "substring_index(element_at(filename, -2), '.' , 1) as month")\
.drop("filename")\
.withColumn("trip_distance", expr("trip_distance * 1.60934"))\
.withColumn("trip_amount", expr("total_amount-tip_amount"))\
.withColumn('payment_type', \
                  when(col("payment_type") == "Credit", 1)\
                  .when(col("payment_type") == "CREDIT", 1)\
                  .when(col("payment_type") == "CASH", 2)\
                  .when(col("payment_type") == "Cash", 2)\
                  .when(col("payment_type") == "No Charge", 3)\
                  .when(col("payment_type") == "NO CHARGE", 3)\
                  .when(col("payment_type") == "Dispute", 4)\
                  .when(col("payment_type") == "DISPUTE", 4)\
                  .when(col("payment_type") == "Unknown", 5)\
                  .when(col("payment_type") == "UNKNOWN", 5)\
                  .when(col("payment_type") == "Voided Trip", 6)\
                  .when(col("payment_type") == "VOIDED TRIP", 6)\
                  .otherwise(expr("payment_type")))\
.select(\
    "year", \
    "month", \
    "pickup_datetime", \
    "dropoff_datetime", \
    "passenger_count", \
    "trip_distance", \
    "payment_type", \
    "tip_amount", \
    "trip_amount", \
    "total_amount" \
).show(2, False)

+----+-----+-------------------+-------------------+---------------+------------------+------------+----------+-----------------+------------+
|year|month|pickup_datetime    |dropoff_datetime   |passenger_count|trip_distance     |payment_type|tip_amount|trip_amount      |total_amount|
+----+-----+-------------------+-------------------+---------------+------------------+------------+----------+-----------------+------------+
|2022|01   |2022-01-05 14:48:04|2022-01-05 14:55:39|1              |1.2874720000000002|1           |1.96      |9.8              |11.76       |
|2022|01   |2022-01-05 13:58:01|2022-01-05 14:05:21|1              |1.448406          |1           |2.9       |9.799999999999999|12.7        |
+----+-----+-------------------+-------------------+---------------+------------------+------------+----------+-----------------+------------+
only showing top 2 rows



## All Data

In [143]:
df = spark.read.parquet(f"/taxi/by_month/*/*.parquet")

In [145]:
all_data = df\
.withColumn("filename", split(input_file_name(), "/"))\
.selectExpr("*", "element_at(filename, -3) as year", "substring_index(element_at(filename, -2), '.' , 1) as month")\
.drop("filename")\
.withColumn("trip_distance", expr("trip_distance * 1.60934"))\
.withColumn("trip_amount", expr("total_amount-tip_amount"))\
.withColumn('payment_type', \
                  when(col("payment_type") == "Credit", 1)\
                  .when(col("payment_type") == "CREDIT", 1)\
                  .when(col("payment_type") == "CASH", 2)\
                  .when(col("payment_type") == "Cash", 2)\
                  .when(col("payment_type") == "No Charge", 3)\
                  .when(col("payment_type") == "NO CHARGE", 3)\
                  .when(col("payment_type") == "Dispute", 4)\
                  .when(col("payment_type") == "DISPUTE", 4)\
                  .when(col("payment_type") == "Unknown", 5)\
                  .when(col("payment_type") == "UNKNOWN", 5)\
                  .when(col("payment_type") == "Voided Trip", 6)\
                  .when(col("payment_type") == "VOIDED TRIP", 6)\
                  .otherwise(expr("payment_type")))\
.select(\
    "year", \
    "month", \
    "pickup_datetime", \
    "dropoff_datetime", \
    "passenger_count", \
    "trip_distance", \
    "payment_type", \
    "tip_amount", \
    "trip_amount", \
    "total_amount" \
)

### Payment Type is still bad... 

In [129]:
all_data = df\
.withColumn("filename", split(input_file_name(), "/"))\
.selectExpr("*", "element_at(filename, -3) as year", "substring_index(element_at(filename, -2), '.' , 1) as month")\
.drop("filename")\
.withColumn("trip_distance", expr("trip_distance * 1.60934"))\
.withColumn("trip_amount", expr("total_amount-tip_amount"))\
.withColumn('payment_type', \
                  when(col("payment_type") == "Credit", 1)\
                  .when(col("payment_type") == "CREDIT", 1)\
                  .when(col("payment_type") == "CRD", 1)\
                  .when(col("payment_type") == "Cre", 1)\
                  .when(col("payment_type") == "CRE", 1)\
                  .when(col("payment_type") == "CASH", 2)\
                  .when(col("payment_type") == "Cash", 2)\
                  .when(col("payment_type") == "CSH", 2)\
                  .when(col("payment_type") == "CAS", 2)\
                  .when(col("payment_type") == "Cas", 2)\
                  .when(col("payment_type") == "No Charge", 3)\
                  .when(col("payment_type") == "NO CHARGE", 3)\
                  .when(col("payment_type") == "NOC", 3)\
                  .when(col("payment_type") == "NO ", 3)\
                  .when(col("payment_type") == "Dispute", 4)\
                  .when(col("payment_type") == "DISPUTE", 4)\
                  .when(col("payment_type") == "DIS", 4)\
                  .when(col("payment_type") == "Dis", 4)\
                  .when(col("payment_type") == "Unknown", 5)\
                  .when(col("payment_type") == "UNKNOWN", 5)\
                  .when(col("payment_type") == "UNK", 5)\
                  .when(col("payment_type") == "Voided Trip", 6)\
                  .when(col("payment_type") == "VOIDED TRIP", 6)\
                  .otherwise(expr("payment_type")))\
.select(\
    "year", \
    "month", \
    "pickup_datetime", \
    "dropoff_datetime", \
    "passenger_count", \
    "trip_distance", \
    "payment_type", \
    "tip_amount", \
    "trip_amount", \
    "total_amount" \
)

In [130]:
all_data.groupby("payment_type").count().show(20)

+------------------+---------+
|      payment_type|    count|
+------------------+---------+
|                 3|  4622574|
|                 5|  1070082|
|                 1|882043526|
|                 4|  1568827|
|                 2|772297398|
|               No |   200451|
|          40.76247|        1|
|40.772744000000003|        1|
|40.748868000000002|        1|
|40.770656000000002|        1|
|40.720982999999997|        1|
|40.780226999999996|        1|
|         40.741132|        1|
|40.760187999999999|        1|
|40.775739000000002|        1|
|40.755504999999999|        1|
|40.768358999999997|        3|
|40.775489999999998|        1|
|40.714882000000003|        1|
|40.795256000000002|        1|
+------------------+---------+
only showing top 20 rows



### Checking Schema

In [146]:
all_data.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- trip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)



year / month should be int as well

In [147]:
all_data = df\
.withColumn("filename", split(input_file_name(), "/"))\
.selectExpr("*", "element_at(filename, -3) as year", "substring_index(element_at(filename, -2), '.' , 1) as month")\
.drop("filename")\
.withColumn("trip_distance", expr("trip_distance * 1.60934"))\
.withColumn("trip_amount", expr("total_amount-tip_amount"))\
.withColumn('payment_type', \
                  when(col("payment_type") == "Credit", 1)\
                  .when(col("payment_type") == "CREDIT", 1)\
                  .when(col("payment_type") == "CRD", 1)\
                  .when(col("payment_type") == "Cre", 1)\
                  .when(col("payment_type") == "CRE", 1)\
                  .when(col("payment_type") == "CASH", 2)\
                  .when(col("payment_type") == "Cash", 2)\
                  .when(col("payment_type") == "CSH", 2)\
                  .when(col("payment_type") == "CAS", 2)\
                  .when(col("payment_type") == "Cas", 2)\
                  .when(col("payment_type") == "No Charge", 3)\
                  .when(col("payment_type") == "NO CHARGE", 3)\
                  .when(col("payment_type") == "NOC", 3)\
                  .when(col("payment_type") == "NO ", 3)\
                  .when(col("payment_type") == "Dispute", 4)\
                  .when(col("payment_type") == "DISPUTE", 4)\
                  .when(col("payment_type") == "DIS", 4)\
                  .when(col("payment_type") == "Dis", 4)\
                  .when(col("payment_type") == "Unknown", 5)\
                  .when(col("payment_type") == "UNKNOWN", 5)\
                  .when(col("payment_type") == "UNK", 5)\
                  .when(col("payment_type") == "Voided Trip", 6)\
                  .when(col("payment_type") == "VOIDED TRIP", 6)\
                  .otherwise(expr("payment_type")))\
.selectExpr(\
    "cast(year as int)", \
    "cast(month as int)", \
    "pickup_datetime", \
    "dropoff_datetime", \
    "passenger_count", \
    "trip_distance", \
    "payment_type", \
    "tip_amount", \
    "trip_amount", \
    "total_amount" \
)

In [148]:
all_data.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- trip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)



### Write results 

In [154]:
!hdfs dfs -rm -r /taxi/raw_all.parquet/

Deleted /taxi/raw_all.parquet


In [155]:
all_data.repartition(55).write.parquet(f"/taxi/raw_all.parquet")

### Stopping Spark 

In [156]:
spark.stop()